This notebook ingests both raw datasets and outputs cleaned csv files. A summary of how the datasets have been changed is as follows:

**Climate Data:**
* The first ~150 records of this dataset were nulls, these have been dropped as they do not make up <1% of the dataset.
* Other nulls have been imputed with the mean of the NULL column in the given month. This code also checks if there is sufficient data in the given month to make such an imputation. In the event that there isn't enough data, previous years' data is included in the calculation.

**Visitation Data:**
* The zeroes in the period of 2020 Weeks 10-14 have been imputed to simulate visitation
* Checks on the validity of this imputation have been included in the EDA notebook

In [25]:
import pandas as pd

# HELPER FUNCTIONS
def climate_impute(df, col):

    """

    This function imputes missing values in a climate dataset. A missing value will
    be imputed with the mean of that column in the given month. If there is not
    enough data for the given month, it will look at the same month in previous years.

    """

    # Add a column which counts nulls per station per month
    null_counts = df.groupby(['Bureau of Meteorology station number','Year','Month'])[col].apply(lambda x: x.isna().sum()).reset_index(name='null_count')
    df = df.merge(null_counts, on=['Bureau of Meteorology station number','Year','Month'], how='left')

    #Case 1: months with <15 nulls → fill with same-year monthly mean
    same_year_mean = df.groupby(['Bureau of Meteorology station number','Year','Month'])[col].transform('mean')

    # Case 2: months with >15 nulls → fill with previous years' monthly mean
    prev_year_mean = (
        df.groupby(['Bureau of Meteorology station number','Month'])[col]
        .transform('mean')  # across all years
    )

    # Manipulate original dataframe, drop helper columns and return
    df[col] = df.apply(
        lambda row: same_year_mean[row.name] if (pd.isna(row[col]) and row['null_count'] < 15)
        else (prev_year_mean[row.name] if pd.isna(row[col]) else row[col]),
        axis=1
    )
    df = df.drop(columns=['null_count'])

    return df

In [26]:
# READ in datasets
climate_df = pd.read_csv('climate_raw.csv')
vis_df = pd.read_csv('visitation_raw.csv')

# Climate Data

In [27]:
climate_df.dtypes

,0
Bureau of Meteorology station number,int64
Year,int64
Month,int64
Day,int64
Maximum temperature (Degree C),float64
Minimum temperature (Degree C),float64
Rainfall amount (millimetres),float64


In [28]:
climate_df.describe()

,Bureau of Meteorology station number,Year,Month,Day,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres)
count,39813.000000,39813.000000,39813.000000,39813.000000,38275.000000,38280.000000,37857.000000
mean,78393.697536,2017.294075,6.429106,15.719263,9.900546,2.754650,4.694667
std,6090.058404,4.499419,3.439861,8.795215,7.118937,5.658537,11.303420
min,71032.000000,2010.000000,1.000000,1.000000,-5.800000,-14.200000,0.000000
25%,71075.000000,2013.000000,3.000000,8.000000,4.000000,-1.700000,0.000000
50%,83024.000000,2017.000000,6.000000,16.000000,9.700000,2.000000,0.000000
75%,83085.000000,2021.000000,9.000000,23.000000,15.500000,7.000000,3.600000
max,85291.000000,2025.000000,12.000000,31.000000,34.000000,22.700000,280.600000


In [29]:
climate_df.shape

(39813, 7)

In [30]:
# CLEAN and save climate data
climate_df = climate_df.iloc[145:] # Drop rows until the 26th of May 2010 (all NULLs)
climate_df = climate_impute(climate_df, 'Maximum temperature (Degree C)')
climate_df = climate_impute(climate_df, 'Minimum temperature (Degree C)')
climate_df = climate_impute(climate_df, 'Rainfall amount (millimetres)')
climate_df.to_csv('climate.csv')

# Visitation Data

In [ ]:
vis_df.dtypes

,0
Year,int64
Week,int64
Mt. Baw Baw,int64
Mt. Stirling,int64
Mt. Hotham,int64
Falls Creek,int64
Mt. Buller,int64
Selwyn,int64
Thredbo,int64
Perisher,int64


In [ ]:
vis_df.describe()

,Year,Week,Mt. Baw Baw,Mt. Stirling,Mt. Hotham,Falls Creek,Mt. Buller,Selwyn,Thredbo,Perisher,Charlotte Pass
count,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000
mean,2019.000000,8.000000,5373.648485,589.981818,20110.333333,24027.145455,29349.296970,5536.533333,30020.884848,39649.515152,2181.812121
std,3.171904,4.333646,3924.269031,529.497558,12198.935589,14176.688461,17768.711945,3383.338655,18363.819945,24258.783674,1334.790842
min,2014.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2016.000000,4.000000,1757.000000,98.000000,8368.000000,8741.000000,12397.000000,2180.000000,11543.000000,15236.000000,842.000000
50%,2019.000000,8.000000,5565.000000,480.000000,21856.000000,28099.000000,33328.000000,6454.000000,34676.000000,46265.000000,2542.000000
75%,2022.000000,12.000000,8513.000000,962.000000,30944.000000,36692.000000,44315.000000,8530.000000,46015.000000,60788.000000,3344.000000
max,2024.000000,15.000000,18031.000000,2097.000000,44648.000000,49697.000000,66326.000000,12863.000000,70634.000000,93226.000000,5033.000000


In [11]:
vis_df.shape

(165, 11)

In [ ]:
# CLEAN and save visitation data

impute_weeks = list(range(10, 15))

# Split out 2020 and other years
df_2020 = vis_df[vis_df['Year'] == 2020].copy()
df_others = vis_df[vis_df['Year'] != 2020].copy()

resort_cols = [c for c in vis_df.columns if c not in ['Year', 'Week']]

for resort in resort_cols:
    # 1. Baseline pattern (mean of other years for weeks 10-14)
    baseline = df_others[df_others['Week'].isin(impute_weeks)].groupby('Week')[resort].mean()

    # 2. Scaling factor (how "low" 2020 was compared to other years overall)
    avg_2020 = df_2020.loc[~df_2020['Week'].isin(impute_weeks), resort].mean()
    avg_others = df_others.loc[~df_others['Week'].isin(impute_weeks), resort].mean()
    scale = avg_2020 / avg_others if avg_others > 0 else 1.0

    # 3. Impute weeks 10-14 of 2020
    for w in impute_weeks:
        if (df_2020['Week'] == w).any():
            vis_df.loc[(vis_df['Year'] == 2020) & (vis_df['Week'] == w), resort] = baseline.loc[w] * scale

vis_df.to_csv('visitation.csv')

/tmp/ipython-input-2990941033.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1645.8463985833198' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  vis_df.loc[(vis_df['Year'] == 2020) & (vis_df['Week'] == w), resort] = baseline.loc[w] * scale
/tmp/ipython-input-2990941033.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '313.60088708752346' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  vis_df.loc[(vis_df['Year'] == 2020) & (vis_df['Week'] == w), resort] = baseline.loc[w] * scale
/tmp/ipython-input-2990941033.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '7808.5706881885635' has dtype incompatible with int64, please explicitly cast to a compatible 